In [ ]:
import paramiko

def init(ip, user, pw, cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    (stdin, stdout, stderr) = cli.exec_command(cmd) #서버에 명령어를 전달하는 메서드
    output_data = stdout.readlines()
    cli.close()
    return output_data
        
ip = "172.16.16.122"
user = "root"
pw = "asd123!@"


def maria_in():
    return """

 # 1️⃣ 필수 패키지 설치
    echo ">>> DNF 플러그인 설치..."
    dnf -y install dnf-plugins-core epel-release expect
    
    # 2️⃣ PHP 리포지토리 등록 및 활성화
    echo ">>> Remi 리포지토리 설정 중..."
    dnf install -y https://rpms.remirepo.net/enterprise/remi-release-9.rpm
    dnf module reset php -y
    dnf module enable php:remi-8.4 -y
    echo "✅ PHP 8.4 모듈 활성화 완료!"
    
    # 3️⃣ Apache + PHP 설치
    echo ">>> Apache + PHP 설치 중..."
    dnf -y install httpd php php-fpm php-mysqlnd php-cli php-common php-devel \
        php-mbstring php-xml php-gd php-json php-intl php-zip php-opcache
    
    systemctl enable --now httpd php-fpm
    echo "✅ Apache & PHP 설치 및 자동 시작 완료!"
    
    # 4️⃣ MariaDB 설치 및 자동 실행
    echo ">>> MariaDB 설치 중..."
    dnf install -y mariadb-server
    systemctl enable --now mariadb
    echo "✅ MariaDB 설치 및 실행 완료!"
    
    # 5️⃣ MariaDB 초기화 (expect 자동화)
    MYSQL_ROOT_PASSWORD="asd123!@"
    
    cat <<'EOF' > /root/mysql_secure.exp
    #!/usr/bin/expect -f
    set timeout -1
    set password [lindex $argv 0]
    
    spawn mysql_secure_installation
    
    expect "Enter current password for root (enter for none):"
    send "\r"
    expect "Switch to unix_socket authentication"
    send "Y\r"
    expect "Change the root password?"
    send "y\r"
    expect "New password:"
    send "$password\r"
    expect "Re-enter new password:"
    send "$password\r"
    expect "Remove anonymous users?"
    send "Y\r"
    expect "Disallow root login remotely?"
    send "Y\r"
    expect "Remove test database and access to it?"
    send "Y\r"
    expect "Reload privilege tables now?"
    send "Y\r"
    expect eof
    EOF
    
    chmod +x /root/mysql_secure.exp
    /usr/bin/expect /root/mysql_secure.exp "$MYSQL_ROOT_PASSWORD"
    echo "✅ MariaDB 초기화 및 비밀번호 설정 완료!"

    """